# Simulating Deletions

In [2]:
import pandas
from time import time

import cobra.test
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

cobra_model = cobra.test.create_test_model("textbook")
ecoli_model = cobra.test.create_test_model("ecoli")

## Knocking out single genes and reactions

A commonly asked question when analyzing metabolic models is what will happen if a certain reaction was not allowed to have any flux at all. This can tested using cobrapy by

In [3]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.reactions.PFK.knock_out()
    print('pfk knocked out: ', cobra_model.optimize())

complete model:  <Solution 0.874 at 0x7f8130c3c760>
pfk knocked out:  <Solution 0.704 at 0x7f8130c3c070>


For evaluating genetic manipulation strategies, it is more interesting to examine what happens if given genes are knocked out as doing so can affect no reactions in case of redundancy, or more reactions if gene when is participating in more than one reaction.

In [4]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.genes.b1723.knock_out()
    print('pfkA knocked out: ', cobra_model.optimize())
    cobra_model.genes.b3916.knock_out()
    print('pfkB knocked out: ', cobra_model.optimize())

complete model:  <Solution 0.874 at 0x7f8131dbc7c0>
pfkA knocked out:  <Solution 0.874 at 0x7f816ddee820>
pfkB knocked out:  <Solution 0.704 at 0x7f8130c3cf40>


## Single Deletions

Perform all single gene deletions on a model

In [5]:
deletion_results = single_gene_deletion(cobra_model)

These can also be done for only a subset of genes

In [6]:
single_gene_deletion(cobra_model, cobra_model.genes[:20])

,ids,growth,status
0,{b0351},0.873922,optimal
1,{b0727},0.858307,optimal
2,{b2587},0.873922,optimal
3,{b0474},0.873922,optimal
4,{b3734},0.374230,optimal
5,{b0726},0.858307,optimal
6,{b3735},0.374230,optimal
7,{b1241},0.873922,optimal
8,{b1276},0.873922,optimal
9,{b1849},0.873922,optimal


This can also be done for reactions

In [7]:
single_reaction_deletion(cobra_model, cobra_model.reactions[:20])

,ids,growth,status
0,{ACALDt},0.873922,optimal
1,{ATPS4r},0.374230,optimal
2,{EX_ac_e},0.873922,optimal
3,{AKGDH},0.858307,optimal
4,{ACKr},0.873922,optimal
5,{ETOHt2r},0.873922,optimal
6,{CS},0.000000,optimal
7,{ALCD2x},0.873922,optimal
8,{ACONTa},0.000000,optimal
9,{D_LACt2},0.873922,optimal


## Double Deletions

Double deletions run in a similar way.

In [8]:
double_gene_deletion(
    cobra_model, cobra_model.genes[-5:]).round(4)

,ids,growth,status
0,{b2935},0.8739,optimal
1,"{b2465, b2464}",0.8739,optimal
2,"{b0008, b2464}",0.8739,optimal
3,"{b0008, b3919}",0.7040,optimal
4,"{b3919, b2465}",0.7040,optimal
5,"{b0008, b2935}",0.8739,optimal
6,{b3919},0.7040,optimal
7,"{b2465, b2935}",0.8739,optimal
8,"{b0008, b2465}",0.8739,optimal
9,"{b3919, b2464}",0.7040,optimal


By default, the double deletion function will automatically use multiprocessing, splitting the task over up to 4 cores if they are available. The number of cores can be manually specified as well. Setting use of a single core will disable use of the multiprocessing library, which often aids debugging.

In [9]:
start = time()  # start timer()
double_gene_deletion(
    ecoli_model, ecoli_model.genes[:25], processes=2)
t1 = time() - start
print("Double gene deletions for 200 genes completed in "
      "%.2f sec with 2 cores" % t1)

start = time()  # start timer()
double_gene_deletion(
    ecoli_model, ecoli_model.genes[:25], processes=1)
t2 = time() - start
print("Double gene deletions for 200 genes completed in "
      "%.2f sec with 1 core" % t2)

print("Speedup of %.2fx" % (t2 / t1))

Double gene deletions for 200 genes completed in 9.02 sec with 2 cores
Double gene deletions for 200 genes completed in 15.48 sec with 1 core
Speedup of 1.72x


Double deletions can also be run for reactions.

In [10]:
double_reaction_deletion(
    cobra_model, cobra_model.reactions[2:7]).round(4)

,ids,growth,status
0,"{ACt2r, ACKr}",0.8739,optimal
1,"{ACONTa, ACKr}",0.0000,optimal
2,"{ACONTa, ACONTb}",0.0000,optimal
3,{ACONTa},0.0000,optimal
4,"{ACONTb, ACt2r}",0.0000,optimal
5,{ACt2r},0.8739,optimal
6,"{ACONTa, ADK1}",0.0000,optimal
7,"{ACONTa, ACt2r}",0.0000,optimal
8,"{ADK1, ACt2r}",0.8739,optimal
9,"{ACONTb, ACKr}",0.0000,optimal


## Accessing individual deletion results

Note that the indices for deletions are python [set](https://docs.python.org/3.8/library/stdtypes.html#set-types-set-frozenset) objects. This is the appropriate type since the order of deletions does not matter. Deleting reaction 1 and reaction 2 will have the same effect as deleting reaction 2 and reaction 1. 

To make it easier to access results all DataFrames returned by COBRAPpy deletion functions have a `knockout` indexer that makes that a bit simpler. Each entry in the indexer is treated as a single deletion entry. So you need to pass sets for double deletions.

In [11]:
single = single_reaction_deletion(cobra_model)
double = double_reaction_deletion(cobra_model)

print(single.knockout["ATPM"])
print(double.knockout[{"ATPM", "TKT1"}])

ids    growth   status
89  {ATPM}  0.916647  optimal
               ids   growth   status
2238  {ATPM, TKT1}  0.90584  optimal


This can be used to get several deletions at once and will also work for Reaction or Gene objects (depending on what you deleted) directly.

In [12]:
atpm = cobra_model.reactions.ATPM
tkt1 = cobra_model.reactions.TKT1
pfk = cobra_model.reactions.PFK

print(single.knockout[atpm, tkt1, pfk])
print(double.knockout[{atpm, tkt1}, {atpm, pfk}, {atpm}])


ids    growth   status
15   {PFK}  0.704037  optimal
17  {TKT1}  0.864759  optimal
89  {ATPM}  0.916647  optimal
               ids    growth   status
762         {ATPM}  0.916647  optimal
2238  {ATPM, TKT1}  0.905840  optimal
2533   {PFK, ATPM}  0.704037  optimal
